In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import ipympl
import glob, os
import gym
import math
import random
from itertools import count

In [2]:
from model import Actor, Critic, OrnsteinUhlenbeckActionNoise
from memory import Transition, ReplayMemory
from train import optimize_model_AC

In [3]:
env = gym.make('MountainCarContinuous-v0')
print(env.action_space, env.action_space.high, env.action_space.low)
print(env.observation_space.shape[0], env.observation_space.high, env.observation_space.low)

Box(1,) [1.] [-1.]
2 [0.6  0.07] [-1.2  -0.07]


In [8]:
BATCH_SIZE = 32
GAMMA = 0.999
LRA=0.0001      #LEARNING RATE ACTOR
LRC=0.001       #LEARNING RATE CRITIC
epsilon = 1
epsilon_decay = 1./1000000 #this is ok for a simple task like inverted pendulum, but maybe this would be set to zero for more
                     #complex tasks like Hopper; epsilon is a decay for the exploration and noise applied to the action is 
                     #weighted by this decay. In more complex tasks we need the exploration to not vanish so we set the decay
                     #to zero.

noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(1))
print(noise())
actor = Actor(env.observation_space.shape[0], 1)
critic = Critic(env.observation_space.shape[0], 1)

q_optimizer = optim.Adam(critic.parameters(), lr=LRA)
policy_optimizer= optim.Adam(actor.parameters(), lr=LRC)

memory = ReplayMemory(10000)
loss_fn = nn.MSELoss().type(torch.FloatTensor)

[0.01914676]


In [ ]:
num_episodes = 1000
step_counter = 0
successes = 0
for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = torch.from_numpy(env.reset()).type(torch.FloatTensor)
    states = []
    rewards = []
    actions = []
    counter = 0
    
    for t in count():
        action = actor(state)
        epsilon -=  epsilon_decay
        action += torch.from_numpy(10*noise()*max(0, epsilon)).type(torch.FloatTensor)
        #print(action, type(action))
        next_state, reward, done, _ = env.step(action.detach().numpy())
        #print(state, type(state))
        #print(next_state, type(next_state))
        reward = torch.tensor([reward]).type(torch.FloatTensor)
        #action = torch.tensor([action]).type(torch.FloatTensor)
        #env.render()
        
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        
        if not done:
            next_state = torch.from_numpy(next_state).type(torch.FloatTensor)
            # Move to the next state
            state = next_state
        else:
            if next_state[0] >= env.goal_position:
                successes += 1
                print(successes)
            next_state = None

        counter += 1
        step_counter += 1
        # Perform one step of the optimization (on the target network)
        optimize_model_AC(actor, critic, q_optimizer, policy_optimizer, 
                              memory, loss_fn, BATCH_SIZE)
        if done:
            discounted_rewards = []
            discounted_rewards.append(0)
            for i in range(len(rewards)):
                discounted_rewards[0] += rewards[i]*(GAMMA**(i))
            for i in range(1, len(rewards)):
                discounted_rewards.append((discounted_rewards[i-1] - rewards[i-1])/GAMMA)
            #print("t = ", t)
            #print("actions = ", actions)
            #print("discounted_rewards = ", discounted_rewards)
            # Store the transitions in memory
            for i in range(len(rewards)):
                memory.push(states[i], actions[i], discounted_rewards[i])
            break
print('Complete')
env.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


In [ ]:
successes = 0
for i_episode in range(100):
    # Initialize the environment and state
    state = torch.from_numpy(env.reset()).type(torch.FloatTensor)
    for t in count():
        # Select and perform an action
        env.render()
        action = actor(state)
        next_state, reward, done, _ = env.step(action.detach().numpy())
        reward = torch.tensor([reward]).type(torch.FloatTensor)
        
        next_state = torch.from_numpy(next_state).type(torch.FloatTensor)
        state = next_state
        
        if done:
            if state[0] >= env.goal_position:
                successes += 1
            break
print(successes/num_episodes*100)
print('Complete')
env.close()